In [17]:
[global]
# The output directory for generated files. 
parameter: cwd = path("output")
# Sample meta data list
parameter: samples = path
# Raw data directory, default to the same directory as sample list
parameter: data_dir = path(f"{samples:d}")
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 8
# Software container option
parameter: container = ""
from sos.utils import expand_size
cwd = path(f'{cwd:a}')

import pandas as pd


In [20]:
[leafcutter_norm_1]
parameter: npc = 10
parameter: chr_blacklist = f'{samples:d}/black_list.txt'
output: f'{cwd}/{samples:bn}_intron_usage_perind.counts.gz.PCs',
        f'{cwd}/{samples:bn}_intron_usage_perind.counts.gz_prepare.sh',
        f'{cwd}/sample_list_intron_usage_phenotype_list_by_chromosome.txt'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
bash: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', container=container
    /opt/leafcutter/scripts/prepare_phenotype_table.py ${_input} -p ${npc} --ChromosomeBlackList ${chr_blacklist}
python: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', container=container
    import pandas as pd
    blk_lst = pd.read_csv(f'${chr_blacklist}', sep = " ", header=None)
    blk_lst = blk_lst.iloc[:,0].values.tolist()
    df = pd.read_csv(f'${_output[1]}', sep=" ")
    df = df[df['bgzip'] == "-p"]
    names = df.iloc[:,2].values.tolist()
    chrs = []
    for i in names:
        i = i.split('_')[-1]
        i = i.split('.')[0]
        chrs.append(i)
    zipped = list(zip(chrs, names))
    dfout = pd.DataFrame(zipped, columns=['#chr','#dir'])
    for i in blk_lst:
        dfout = dfout[dfout['#chr'] != str(i)]
    dfout.to_csv(f'${cwd}/sample_list_intron_usage_phenotype_list_by_chromosome.txt', sep="\t", index=None)

In [ ]:
[leafcutter_norm_2]
molecular_pheno_chr_inv = pd.read_csv(f'{_input[2]}',sep = "\t")
molecular_pheno_chr_inv = molecular_pheno_chr_inv.values.tolist()
file_inv = [x[1][:-3] for x in molecular_pheno_chr_inv]
input: file_inv # This design is necessary to avoid using for_each, as sos can not take chr number as an input.
output: f'{_input[0]:n}.qqnorm.txt'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container
    head -1 ${_input[0]:r}  > ${_output}
    cat ${_input:r} | grep -v "#Chr" >> ${_output}